# Atividade 1 - Prado

Gustavo Coelho - GU3054829

In [30]:
import pandas as pd

## Etapa 1 – Importação e visualização inicial

1. Carregue o CSV no Colab com pandas.read_csv().
2. Use .head() e .info() para visualizar os dados.
3. Verifique se há dados faltantes ou inconsistentes.


In [31]:
df = pd.read_csv("01. academia_dataset.csv")

df.head()

,aluno_id,sexo,idade,plano,valor_mensal,tempo_meses,academia
0,100,M,19,Premium,149.9,9,Unidade B
1,101,M,26,Premium,149.9,4,Unidade E
2,102,M,55,Intermediário,119.9,2,Unidade A
3,103,M,31,Básico,89.9,17,Unidade E
4,104,M,53,Básico,89.9,23,Unidade E


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   aluno_id      100 non-null    int64  
 1   sexo          100 non-null    object 
 2   idade         100 non-null    int64  
 3   plano         100 non-null    object 
 4   valor_mensal  100 non-null    float64
 5   tempo_meses   100 non-null    int64  
 6   academia      100 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 5.6+ KB


Não há dados faltantes!

## Etapa 2 – Agrupamento por unidade

Objetivo: Descobrir qual unidade tem mais alunos e maior receita
1. Agrupe por academia usando .groupby().
2. Gere uma tabela com:
- Quantidade de alunos
- Receita total (valor_mensal * tempo_meses)

In [33]:
df_por_academia = df.groupby("academia").size()

print("Quantidade de alunos por academia:")

df_por_academia

Quantidade de alunos por academia:


academia
Unidade A    17
Unidade B    20
Unidade C    21
Unidade D    22
Unidade E    20
dtype: int64

In [76]:
df["receita_total"] = df["valor_mensal"] * df["tempo_meses"]

df_receita_academia = df.groupby("academia")["receita_total"].sum().reset_index()

df_receita_academia.sort_values(by="receita_total", ascending=False, inplace=True)

df_receita_academia

,academia,receita_total
2,Unidade C,34020.5
4,Unidade E,30154.0
3,Unidade D,28564.0
1,Unidade B,25269.1
0,Unidade A,24758.9


Tirando uma prova real dos valores acima

In [35]:
df.loc[df["academia"] == "Unidade A", "receita_total"].sum()

np.float64(24758.9)

# Etapa 3 – Análise por plano de assinatura
Objetivo: Avaliar o desempenho de cada tipo de plano
1. Agrupe por plano e gere:
- Número de alunos por plano
- Receita total por plano
- Tempo médio de permanência



In [37]:
df_por_plano = (
    df.groupby("plano")
    .agg({"aluno_id": "count", "receita_total": "sum", "tempo_meses": "mean"})
    .rename(
        columns={
            "aluno_id": "quantidade_alunos",
            "receita_total": "total_receita",
            "tempo_meses": "media_tempo_permanencia",
        }
    )
)

print("Dados por plano:")
df_por_plano

Dados por plano:


,quantidade_alunos,total_receita,media_tempo_permanencia
plano,,,
Básico,41,47916.7,13.000000
Intermediário,33,41485.4,10.484848
Premium,26,53364.4,13.692308


In [78]:
df_por_plano["valor_medio_plano"] = df_por_plano["total_receita"] / df_por_plano["quantidade_alunos"]

print("Ranking de planos por valor médio (mais lucrativos):")
df_por_plano[["valor_medio_plano"]].sort_values(by="valor_medio_plano", ascending=False)

Ranking de planos por valor médio (mais lucrativos):


,valor_medio_plano
plano,
Premium,2052.476923
Intermediário,1257.133333
Básico,1168.700000


# Etapa 4 – Conclusão e geração de insights
Objetivo: Criar uma conclusão com base nos dados agrupados
1. Destaque qual plano e qual unidade geram mais receita.
2. Identifique o perfil médio dos alunos mais fiéis.
3. Responda: há alguma unidade que parece ter desempenho fraco?


In [60]:
df_por_plano_academia = (
    df.groupby(["academia", "plano"])
    .agg({"receita_total": "sum"})
    .rename(
        columns={
            "receita_total": "total_receita",
        }
    )
    .sort_values(
        by=["total_receita"],
        ascending=[False],
    )
)

df_por_plano_academia

total_receita
academia  plano                       
Unidade E Premium              13940.7
Unidade D Básico               12406.2
Unidade C Intermediário        11990.0
Unidade E Básico               11417.3
Unidade B Premium              11242.5
Unidade C Premium              11242.5
          Básico               10788.0
Unidade A Premium               9293.8
Unidade D Intermediário         8512.9
Unidade A Intermediário         8273.1
Unidade B Intermediário         7913.4
Unidade D Premium               7644.9
Unidade A Básico                7192.0
Unidade B Básico                6113.2
Unidade E Intermediário         4796.0

A unidade e plano que mais geram receita é a Unidade E com o Plano Premium

In [44]:
df_alunos_mais_fieis = df[df["tempo_meses"] == df["tempo_meses"].max()]

print("Alunos mais fiéis:")
df_alunos_mais_fieis

Alunos mais fiéis:


,aluno_id,sexo,idade,plano,valor_mensal,tempo_meses,academia,receita_total
43,143,M,52,Básico,89.9,24,Unidade E,2157.6
46,146,F,44,Intermediário,119.9,24,Unidade A,2877.6
47,147,M,21,Intermediário,119.9,24,Unidade C,2877.6
55,155,F,45,Premium,149.9,24,Unidade E,3597.6
96,196,F,60,Básico,89.9,24,Unidade C,2157.6


In [79]:
media_numericos = df_alunos_mais_fieis.select_dtypes(include="number").mean()
moda_ncategoricos = df_alunos_mais_fieis.select_dtypes(exclude="number").mode().iloc[0]
perfil_medio_aluno_fiel = pd.concat([media_numericos, moda_ncategoricos])

print("Perfil médio do aluno mais fiel:")
display(perfil_medio_aluno_fiel)


Perfil médio do aluno mais fiel:


aluno_id             157.4
idade                 44.4
valor_mensal         113.9
tempo_meses           24.0
receita_total       2733.6
sexo                     F
plano               Básico
academia         Unidade C
dtype: object

In [71]:
df_receita_academia.min()

academia         Unidade A
receita_total      24758.9
dtype: object

A unidade que apresenta menor receita é a "Unidade A"

# Mini relatório

1. Unidades com melhor desempenho:  
    As unidades com melhor desempenho monetário foram as:
    - Unidade C	- Receita R$34020,5
    - Unidade E	- Receita R$30154
    - Unidade D - Receita R$28564

2. Planos mais lucrativos:  
    O plano mais lucrativo é Premium, que possui o menor número de usuários (26), e maior receita (R$53.364,4) tendo assim uma receita média total de R$2.052,47 por aluno.

3. Perfil do aluno mais fiel:  
    O perfil de aluno mais fiel é:
    - Idade: 44.4
    - Valor_mensal: 113.9
    - Tempo_meses: 24.0
    - Receita_total: 2733.60
    - Sexo: F
    - Plano: Básico
    - Academia: Unidade C

4. Principais aprendizados com agrupamento de dados:  
    - O plano básico é o que tem o maior número de alunos, porém é o que gera menos receita total
    - Os 3 planos são mais eficientes nessas unidades, respectivamente:
      - Premium: Unidade E
      - Intemediário: Unidade C
      - Básico: Unidade D
  
5. Sugestões para a rede de academias:  
   - Estudar a implementação de um aumento no plano básico, onde o maior número de alunos estão concentrando, visando receita, começando pela unidade B onde esse plano tem menos público
